# Analysis of Movie Industry


insert pretty picture here, sponsored by microsoft, microsoft logo

## Overview

In light of Microsoft's desire to expand their enterprise into the movie sector, we have analyzed various qualities of movie data to help develop some insights to help maximize Microsoft's ROI for their potential startup. 

{PUT SUMMARY OF INSIGHTS HERE}

Since we are both relatively unfamiliar with the business end of the industry, supplementary research on the working of the industry as a whole were considered to help drive and focus our analysis. Access sources HERE.

Data was analyzed from IMDB, Box Office Mojo, and The-Numbers.com . (include hyperlinks)
From each of the above 

## Business Problem

picture?

Highlight specific areas of focus
- start up company
- changing industry
- good strategies {use movie terms: release strategy, dump month, etc}

## Unpacking the Data

*Recognize credentials of sources listed above as credible, good data supply, etc



In [5]:
import pandas as pd

We adjusted each of the monetary values to accomodate for inflation. [Here](https://github.com/datadesk/cpi#:~:text=Working%20with%20Python,the%20Bureau%20of%20Labor%20Statistics) is the link to the package info.

In [ ]:
# if you do not have this cpi package, you can download as:
# pip install cpi (!pip install cpi if you'd like in your Jupyter notebook)

In [7]:
import cpi
cpi.update() # run to make sure current Consumer Price Index is being used in calculations

In [8]:
########## JAMIE CLEANING CODE ###########

In [12]:
bomMovieGross = pd.read_csv('data/bom.movie_gross.csv.gz')

imdbNameBasics = pd.read_csv('data/imdb.name.basics.csv.gz')

imdbTitleAkas = pd.read_csv('data/imdb.title.akas.csv.gz')

imdbTitleBasics = pd.read_csv('data/imdb.title.basics.csv.gz')

imdbTitleCrew = pd.read_csv('data/imdb.title.crew.csv.gz')

imdbTitlePrincipals = pd.read_csv('data/imdb.title.principals.csv.gz')

imdbTitleRatings = pd.read_csv('data/imdb.title.ratings.csv.gz')

rtMovieInfo = pd.read_csv('data/rt.reviews.tsv.gz', sep='\t', encoding='windows-1252')

rtReviews = pd.read_csv('data/rt.reviews.tsv.gz', sep='\t', encoding='windows-1252')

tmdbMovies = pd.read_csv('data/tmdb.movies.csv.gz')

tnMovieBudgets = pd.read_csv('data/tn.movie_budgets.csv.gz')

#### kaggle data csv read here ####

### Cleaning Movie Budget data from [The-Numbers.com](https://www.the-numbers.com/)

In [13]:
tnMovieBudgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


The production budget, domestic gross, and worldwide gross columns need to be converted into numbers:

In [ ]:
money_cols = ['worldwide_gross', 'domestic_gross', 'production_budget']
tnMovieBudgets[money_cols] = tnMovieBudgets[money_cols].applymap(lambda x: int(x.replace('$','').replace(',','')))

In [25]:
tnMovieBudgets[money_cols].head(1)

,worldwide_gross,domestic_gross,production_budget
0,2776345279,760507625,425000000


Then, we adjusted the dates to create separate columns for month and year released:

In [23]:
tnMovieBudgets['release_date'] = pd.to_datetime(tnMovieBudgets['release_date'], errors='coerce') # what does coerce do?
tnMovieBudgets['release_date']

tnMovieBudgets['month_released'] = tnMovieBudgets['release_date'].apply(lambda x: str(x))
tnMovieBudgets['month_released'] = tnMovieBudgets['month_released'].apply(lambda x: int(x[5:7]))

tnMovieBudgets['year'] = tnMovieBudgets['release_date'].apply(lambda x: str(x))
tnMovieBudgets['year'] = tnMovieBudgets['year'].apply(lambda x: int(x[:4]))

tnMovieBudgets.drop(labels='release_date', axis=1, inplace=True)

In [26]:
tnMovieBudgets[['month_released', 'year']].head(1)

,month_released,year
0,12,2009


### Cleaning [Box Office Mojo](https://www.boxofficemojo.com/) data

In [27]:
bomMovieGross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [ ]:
bomMovieGross['foreign_gross'] = bomMovieGross['foreign_gross'].apply(lambda x: float(str(x).replace(',','')))

### Adjusting tnMovieBudgets and bomMovieGross for inflation

First, we made sure all NaN values were an integer:

In [ ]:
bomMovieGross['foreign_gross'].fillna(value=0, inplace=True)



Then, we converted the numbers using **cpi.inflate**:

In [ ]:
tnMovieYA = tnMovieBudgets[tnMovieBudgets['year']<2019]

tnMovieBudgets['domestic_adj'] = tnMovieYA.apply(lambda x: cpi.inflate(x.domestic_gross, x.year), axis=1)

tnMovieBudgets['worldwide_adj'] = tnMovieYA.apply(lambda x: cpi.inflate(x.worldwide_gross, x.year), axis=1)

tnMovieBudgets['budget_adj'] = tnMovieYA.apply(lambda x: cpi.inflate(x.production_budget, x.year), axis=1)

In [293]:
bomMovieGross['domestic_adj'] = bomMovieGross.apply(lambda x: cpi.inflate(x.domestic_gross, x.year), axis=1)

bomMovieGross['foreign_adj'] = bomMovieGross.apply(lambda x: cpi.inflate(x.foreign_gross, x.year), axis=1)

To account for data that was NOT adjusted (the CPI does not inflate any data that came after 2018), we filled the NaNs with their original numbers:

In [300]:
tnMovieBudgets[money_cols].fillna(value=tnMovieBudgets['domestic_gross'], inplace=True)

In [ ]:
bom_money_cols = ['worldwide_gross', 'domestic_gross']
tnMovieBudgets[bom_money_cols].fillna(value=tnMovieBudgets['domestic_gross'], inplace=True)

In [294]:
# move to merging section
bomMovieGross.rename(mapper={'title' : 'primary_title'}, axis=1, inplace=True)

In [295]:
bomMovieGross.head()

,primary_title,studio,domestic_gross,foreign_gross,year,domestic_adj,foreign_adj
0,Toy Story 3,BV,415000000.0,652000000.0,2010,4.925641e+08,7.738598e+08
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010,3.966625e+08,8.205050e+08
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010,3.513229e+08,7.884587e+08
3,Inception,WB,292600000.0,535700000.0,2010,3.472874e+08,6.358231e+08
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010,2.833134e+08,6.099487e+08


In [ ]:
tnMovieBudgets.head()

After some cursory analysis, we realized that we wanted to measure averages of domestic and worldwide gross **combined**, so we created a new column, 'total_gross':

In [ ]:
tnMovieBudgets['total_gross'] = tnMovieBudgets['worldwide_adj'] + tnMovieBudgets['domestic_adj']

In [ ]:
bomMovieGross['total_BOM_gross'] = bomMovieGross['foreign_adj'] + bomMovieGross['domestic_adj']

### Cleaning [The Movie DataBase](https://www.themoviedb.org/) Data

In [ ]:
tmdbMovies.head()

Drop some unnecessary columns:

In [ ]:
dropcols = ['Unnamed: 0', 'original_title', 'original_language']

tmdbMovies.drop(dropcols, axis=1, inplace=True)

### Cleaning [IMDb](https://www.imdb.com/) Data

In [ ]:
imdbNameBasics.head()

In [ ]:
imdbTitleBasics.head()

In [ ]:
imdbTitleBasics.head()

In [ ]:
imdbTitleCrew.head()

In [ ]:
imdbTitlePrincipals.head()

In [ ]:
imdbTitleRatings.head()

In [ ]:
imdbTitleAkas.head()

### Merging the data

Since we want to merge across the **'tconst'** column, we have to rename **'title_id'** of imdbTitleAkas:

In [ ]:
imdbTitleAkas.rename(mapper={'title_id' : 'tconst'}, axis=1, inplace=True)

First, we are going to merge all of the imdb data together on the column 'tconst':

In [ ]:
imdb_dfs = [imdbTitleBasics, imdbTitleAkas, imdbTitleCrew, imdbTitlePrincipals, imdbTitleRatings]

In [ ]:
# maybe clean this up later with a for loop?
imdb_master = pd.merge(
                    pd.merge(
                            pd.merge(
                                    pd.merge(
                                        imdb_dfs[0], imdb_dfs[1], on='tconst', how = 'outer'
                                        ), 
                                imdb_dfs[2], on='tconst', how = 'outer'
                                ), 
                        imdb_dfs[3], on='tconst', how = 'outer', suffixes=('_akas', '_princ')
                        ),
                  imdb_dfs[4], on='tconst', how = 'outer')

We found some weird data, so we made sure to get rid of any movies whose 'start_year' was after 2021:

In [ ]:
imdb_master.loc[imdb_master['start_year'] >= 2021]

In [ ]:
imdb_master = imdb_master.loc[imdb_master['start_year'] <= 2021]

We also removed any movie duplicates by their 'tconst' id:

In [ ]:
imdb_master.drop_duplicates(subset=['tconst'], inplace=True)

**Merging in tmdbMovies to IMDb_master by 'primary_title':**

Before merging, we had to rename the 'title' column 'primary_title' so we could merge on that column.

In [ ]:
tmdbMovies.rename(mapper={'title' : 'primary_title'}, axis=1, inplace=True)

In [328]:
imdb_master = pd.merge(imdb_master, tmdbMovies, on='primary_title', how = 'outer')

**Making IMDB_master with BOM GROSS**

In [28]:
imdb_BOM = pd.merge(imdb_master, bomMovieGross, on='primary_title', how='outer')

Filled all NaN values with 'Unknown':

In [ ]:
imdb_BOM.fillna(value='Unknown', inplace=True)

Looking at the 'genres' column, we need to make the single string into a list of multiple genres:

In [ ]:
imdb_BOM['genres'] = imdb_BOM['genres'].apply(
    lambda x: x.split(',') if (',' in x) else ([x] if (x != 'Unknown') else x)
    )

**Making IMDb_master with tnMovieBudgets**

In [361]:
tnMovieBudgets.rename(mapper={'movie' : 'primary_title'}, axis=1, inplace=True)

In [534]:
tnMoviesAdj = tnMovieBudgets[['primary_title', 'month_released', 'year', 'domestic_adj', 'worldwide_adj', 'budget_adj', 'total_gross']]

In [535]:
imdb_master_money = pd.merge(imdb_master, tnMoviesAdj, on='primary_title')

In [ ]:
imdb_master_money.fillna(value='Unknown', inplace=True)

In [ ]:
imdb_master_money['genres'] = imdb_master_money['genres'].apply(
    lambda x: x.split(',') if (',' in x) else ([x] if (x != 'Unknown') else x)
    )

**Making IMDb_master with BOTH tnMovieBudgets and bomMovieGross**

In [340]:
# Merge Movie Budgets onto imdb_BOM
imdb_BOM_tn = pd.merge(imdb_BOM, tnMoviesAdj, on='primary_title')

In [341]:
imdb_BOM_tn.fillna(value='Unknown', inplace=True)

### Making a list of genres, and a genre dictionary

Upon initial observation of the data, we knew we wanted to see if we could find any patterns related to genre. This required us to create a list of all the unique genres, as well as a **dictionary** whose keys were the genre names, and whose values were the DataFrame with all the movies containing that genre.

In [ ]:
genres = []
for movie in imdb_BOM_tn['genres']:
    if type(movie)==list:
        for genre in movie:
            if genre not in genres:
                genres.append(genre)
 

In [ ]:
print(genres)

We found that the 'Reality-TV' genre doesn't have much data to offer:

In [ ]:
len(imdb_master_money['genres']['Reality-TV' in imdb_master_money['genres']])

So we'll delete 'Reality-TV' from genres:

In [ ]:
genres.pop(-1)

We created a function called **genre_contained()** that takes in a single genre and a DataFrame, and returns a list of indices that can be used to locate all the movies that contain that genre.

In [ ]:
def genre_contained(genre, df):
    index_list = []
    ctr = 0
    for row in df['genres']:
        if type(row) == list:
            for i in row:
                if i == genre:
                    index_list.append(ctr) # should i add continue here          
        ctr += 1
    return index_list

Now, we can update genre_dict with all the info for each genre!

In [ ]:
genre_dict = {}

for genre in genres:
    genre_dict[genre] = imdb_master_money.iloc[genre_contained(genre, imdb_master_money)]

We made **2** other dictionaries for imdb_BOM_tn, and imdb_BOM_tn with some additional genre "combos":

In [ ]:
for genre in genres:
    genre_dict_2[genre] = imdb_BOM_tn.iloc[genre_contained(genre, imdb_BOM_tn)]

To create the **3rd** genre dictionary, we had to create a function **genre_contained_combos()** which would take in a dataframe and return a dictionary of the additional genres and their respective data.

In [ ]:
def genre_contained_combos(df):
    combo_dict = {}
    
    combo_list = [("Romance","Comedy"), ("Animation","Comedy"), ("Adventure","Comedy"), 
                  ("Adventure","Drama"), ("Romance","Drama"), ("Animation", "Drama"), 
                 ("Family", "Comedy"), ("Family", "Drama")]
    
    for k in range(0, len(combo_list)):
        pair_name = combo_list[k][0] + '/' + combo_list[k][1]
        index_list = []
        ctr = 0
        rel_genres = ['Romance', 'Comedy', 'Animation', 'Drama', 'Adventure', 'Family']
        for row in df['genres']:
            if type(row) == list:
                row_pair_keepr = []
                for i in row:
                    #print(row)
                    if i in rel_genres:
                        row_pair_keepr.append(i)
                        if len(row_pair_keepr) == 2:
                            if (row_pair_keepr[0] in combo_list[k]) & (row_pair_keepr[1] in combo_list[k]):
                                index_list.append(ctr)                 
            ctr += 1
        combo_dict[pair_name] = imdb_BOM_tn.iloc[index_list]
        
    return combo_dict

In [ ]:
genre_dict_3 = genre_contained_combos(imdb_BOM_tn)
genre_dict_3.update(genre_dict_2)

Here are some variations the genre data 

In [408]:
# top 15 genres by their frequency in the DataFrame (how many movies classified with this descriptor)
top_15_genres = {i:len(genre_dict_2[i]) for i in genres}
sorted_top_15_genres = sorted(top_15_genres.items(), key=lambda kv: kv[1], reverse=True)[:15]
sorted_top_15_genres

[('Drama', 1817),
 ('Comedy', 842),
 ('Action', 710),
 ('Thriller', 594),
 ('Documentary', 499),
 ('Adventure', 490),
 ('Horror', 415),
 ('Crime', 398),
 ('Romance', 372),
 ('Mystery', 249),
 ('Biography', 248),
 ('Sci-Fi', 229),
 ('Family', 204),
 ('Fantasy', 202),
 ('Animation', 148)]

In [409]:
# split above tuple list into lists
top_genre_ct_names = [name[0] for name in sorted_top_15_genres]
top_genre_cts = [name[1] for name in sorted_top_15_genres]

In [410]:
# measuring top genres by popularity column

top_genres_popularity = {i:genre_dict_2[i]['popularity'][genre_dict_2[i]['popularity']!='Unknown'].mean() for i in genres}
sorted_top_genres_popularity = sorted(top_genres_popularity.items(), key=lambda kv: kv[1], reverse=True)
sorted_top_genres_popularity

[('Adventure', 15.849670644391395),
 ('Sci-Fi', 14.345203124999998),
 ('Fantasy', 14.015947674418602),
 ('Action', 13.435757024793395),
 ('Animation', 13.135606299212599),
 ('Musical', 10.57488),
 ('Crime', 10.242686956521734),
 ('Comedy', 9.783019774011308),
 ('Mystery', 9.461259999999992),
 ('News', 9.405199999999999),
 ('Biography', 9.351870813397127),
 ('Family', 9.19519230769231),
 ('Western', 8.6081),
 ('Thriller', 8.605823170731703),
 ('History', 8.564258823529412),
 ('Music', 8.364701492537312),
 ('Romance', 8.276813504823155),
 ('Drama', 8.089868055555574),
 ('Horror', 7.978852691218126),
 ('Sport', 6.140416666666668),
 ('War', 5.647642857142857),
 ('Documentary', 5.6210893371757935)]

In [ ]:
most_popular_names = [name[0] for name in sorted_top_genres_popularity][:15]
most_popular_avgs = [avg[1] for avg in sorted_top_genres_popularity][:15]

print(most_popular_names)

In [ ]:
########### MOOMEN CLEANING CODE #############

### Relevant Variables

Explain each data set

In [3]:
# show head of dataset 1

In [ ]:
# show head of dataset 2

In [ ]:
# show head of dataset 3

In [ ]:
# show head of dataset 4

In [ ]:
# show head of dataset 5

In [ ]:
# show head of dataset 6

## Cleaning / Reorganizing Data

Summarize cleaning process

In [ ]:
################ JAMIE CLEANING #################

In [ ]:
################ CHRISTOS CLEANING ################

### Sub-elements of cleaning (such as important functions)

picture to divide the two sections

## Analysis

Analysis is split into four sections:
1. Analyzing Budget vs. Total Gross
2. Analyzing Genre type vs. Total Gross
3. Analyzing Release Month vs. Total Gross
4. Christos viz

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

### 1. Analyzing Budget vs. Total Gross

We wanted to see if there was a strong enough correlation between BUDGET and TOTAL GROSS.

TOTAL GROSS is the sum of DOMESTIC GROSS and WORLDWIDE GROSS.

In [ ]:
import seaborn as sns
from scipy import stats

For the y values (total gross values), we found that there were **old** movies (such as the the original release of Snow White in 1937) that were proving to be outliers. We removed those items from the dataset.

In [ ]:
budget_nums = list(imdb_master_money[imdb_master_money['total_gross']<3000000000]['budget_adj'].values)
total_gross_nums = list(imdb_master_money[imdb_master_money['total_gross']<3000000000]['total_gross'].values)

slope, intercept, r_value, p_value, std_err = stats.linregress(budget_nums, total_gross_nums)

ax = sns.regplot(x=budget_nums, y=total_gross_nums, scatter_kws={"color": "mediumseagreen"}, line_kws={"color": "black", "label":"y={0:.1f}x+{1:.1f}".format(slope,intercept)})
ax.set(xlabel="Budget (Billions)", ylabel = "Total Gross (Billions)")
ax.set_title('Relation between Budget and Total Gross (with corresponding regression line)', size=12)

ax.legend()

The **r** value will tell us the strength of the correlation:

In [ ]:
# for PPT, point out strong correlation
print(r_value)

We found the correlation between BUDGET and TOTAL GROSS to be substantial enough to conclude that ...

### 2. Analyzing Genre type vs. Total Gross

### 3. Analyzing Release Month vs. Total Gross

### Analysis of Variable #4

In [ ]:
################ CHRISTOS VISUALIZATIONS ####################


### Analysis of Variable #1

desc here

### Analysis of Variable #2

### Analysis of Variable #3

### Analysis of Variable #4

# Conclusions

summaries in nice bullet points 

# Next Steps

next steps